In [1]:
from ibl_pipeline.analyses import behavior
from ibl_pipeline import behavior as behavior_ingest
from ibl_pipeline import subject, acquisition
from ibl_pipeline.utils import psychofit as psy
from uuid import UUID
import numpy as np
import datetime
import datajoint as dj
import json
import seaborn as sns
import pandas as pd
import plotly
from plotly import tools
import plotly.graph_objs as go
import statsmodels.stats.proportion as smp
plotly.offline.init_notebook_mode(connected=True)

Connecting shan@datajoint.internationalbrainlab.org:3306


In [33]:
subj = subject.Subject & 'subject_nickname="CSHL_007"'
# get fit parameters by date
fit_pars = (behavior.BehavioralSummaryByDate.PsychResults * subj).proj(
    'session_date', 'prob_left','threshold', 'bias', 'lapse_low', 'lapse_high').fetch(as_dict=True)
fit_pars = pd.DataFrame(fit_pars)

# get the first date when animal became "trained" and "ready for ephys"
first_trained = subj.aggr(
    behavior.SessionTrainingStatus & 'training_status = "trained"', 
    first_session='DATE(min(session_start_time))')
first_biased = subj.aggr(
    behavior.SessionTrainingStatus & 'training_status = "ready for ephys"', 
    first_session='DATE(min(session_start_time))')

In [34]:
if len(first_trained):
    first_trained_date = first_trained.fetch1('first_session').strftime('%Y-%m-%d')

if len(first_biased):
    first_biased_date = first_biased.fetch1('first_session').strftime('%Y-%m-%d')

In [76]:
yranges = [[0, 100], [-100, 100], [0, 1], [0, 1]]
par_names = ['threshold', 'bias', 'lapse_low', 'lapse_high']

pars = dict()
for par_name in par_names:
    pars[par_name] = []
    
prob_lefts = fit_pars['prob_left'].unique()
    
for iprob_left, prob_left in enumerate(prob_lefts):
    prob_left_filter = fit_pars['prob_left']==prob_left
    if prob_left == 0.2:
        dot_color = 'orange'
    elif prob_left == 0.5:
        dot_color = 'black'
    elif prob_left == 0.8:
        dot_color = 'cornflowerblue'
    else:
        dot_color = 'gray'
    
    fit_pars_sub = fit_pars[prob_left_filter]
    
    for ipar, par_name in enumerate(par_names):
        if ipar==0:
            show_legend = True
        else:
            show_legend = False
        pars[par_name].append(
            go.Scatter(      
                x=[t.strftime('%Y-%m-%d') for t in fit_pars_sub['session_date'].tolist()],
                y=fit_pars_sub[par_name].tolist(),
                mode='markers',
                marker=dict(
                    size=5,
                    color=dot_color),
                name=f'p_left = {prob_left}',
                xaxis='x{}'.format(4-ipar),
                yaxis='y{}'.format(4-ipar),
                showlegend=show_legend,
                legendgroup='p_left'
            ))

In [80]:
pars_data = [pars[par_name][i] for i, prob_left in enumerate(prob_lefts)
             for par_name in par_names]

for ipar, par_name in enumerate(par_names): 
    if ipar==0:
        show_legend = True
    else:
        show_legend = False
        
    if len(first_trained):
        first_trained_plot=go.Scatter(
            x=[first_trained_date, first_trained_date],
            y=yranges[ipar],
            mode="lines",
            marker=dict(
                color='orange'),
            name='first day got trained',
            xaxis='x{}'.format(4-ipar),
            yaxis='y{}'.format(4-ipar),
            showlegend=show_legend,
            legendgroup='date'
        )
        pars_data.append(first_trained_plot)

    if len(first_biased):
        first_biased_plot=go.Scatter(
            x=[first_biased_date, first_biased_date],
            y=yranges[ipar],
            mode="lines",
            marker=dict(
                color='green'),
            name='first day got biased',
            xaxis='x{}'.format(4-ipar),
            yaxis='y{}'.format(4-ipar),
            showlegend=show_legend,
            legendgroup='date'
        )
        pars_data.append(first_biased_plot)


layout = go.Layout(
    xaxis1=dict(
        domain=[0, 1],
        title='Date'
    ),
    yaxis1=dict(
        domain=[0, 0.2],
        anchor='x1',
        range=[-0.02, 1.02],
        title='$Lapse high\ (\\lambda)$'
    ),
    xaxis2=dict(
        domain=[0, 1],
    ),
    yaxis2=dict(
        domain=[0.25, 0.45],
        anchor='x2',
        range=[-0.02, 1.02],
        title='$Lapse low\ (\\gamma)$'
    ),
    xaxis3=dict(
        domain=[0, 1],  
    ),
    yaxis3=dict(
        domain=[0.5, 0.7],
        anchor='x3',
        range=[-105, 105],
        title='$Bias\ (\\mu)$'
    ),
    xaxis4=dict(
        domain=[0, 1],
    ),
    yaxis4=dict(
        domain=[0.75, 1],
        anchor='x4',
        range=[-5, 105],
        title='$Threshold\ (\\sigma)$'
    ),
    height=1000, 
    width=500,
    title='Fit Parameters',
    legend=dict(
        x=0, 
        y=1.05,
        orientation='h')
)

fig = go.Figure(data=pars_data, layout=layout)

plotly.offline.iplot(fig)


In [53]:
f = open('fit_pars.json', 'w')
s = json.dumps(fig.to_plotly_json())
f.write(s)
f.close()

In [67]:
pars_data

[Scatter({
     'marker': {'color': 'black', 'size': 5},
     'mode': 'markers',
     'name': 'p_left = 0.5',
     'showlegend': True,
     'x': [2019-03-04, 2019-03-05, 2019-03-06, 2019-03-11, 2019-03-12, 2019-03-13,
           2019-03-14, 2019-03-15, 2019-03-18, 2019-03-19, 2019-03-25, 2019-03-28,
           2019-03-29, 2019-04-01, 2019-04-02, 2019-04-03, 2019-04-04, 2019-04-05,
           2019-04-08, 2019-04-09, 2019-04-10, 2019-04-11, 2019-04-12, 2019-04-15,
           2019-04-16, 2019-04-17, 2019-04-18, 2019-04-19, 2019-04-22, 2019-04-23,
           2019-04-24, 2019-04-25, 2019-04-29, 2019-04-30, 2019-05-01, 2019-05-02,
           2019-05-03, 2019-05-06, 2019-05-07, 2019-05-09, 2019-05-10, 2019-05-13,
           2019-05-14, 2019-05-15, 2019-05-16, 2019-05-17, 2019-05-20, 2019-05-21,
           2019-05-22, 2019-05-23, 2019-05-24, 2019-05-28, 2019-05-29, 2019-05-30],
     'xaxis': 'x4',
     'y': [13.2759, 4.75779, 9.04783, 5.23084, 8.95637, 18.4227, 44.0027, 34.0027,
           8.1